# Machine Learning avancé : Deep Learning for NLP
On va mettre au point nos premiers modèles Deep Learning appliqué au traitement du langage. Plutôt que de tout recoder de zéro, on va ici s'appuyer sur la puissance de la librairie Keras qui nous permettra de connecter les layers à la volée et d'implémenter des architectures plus exotiques.

Après cela vous saurez:
- utiliser un embedding pré-calculé
- construire un réseau de neurones avec Keras
- construire une architecture custom avec Keras

Cet exercice constitue également le test noté du cours

## Import des Librairies

In [1]:
import data_utils.utils as du
import data_utils.pos as pos 
import numpy as np
import pandas
data='data/'
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, Masking
from keras.optimizers import RMSprop
from keras import backend as K

## Présentation du problème
Le POS-Tagging et le Shallow Parsing constituent deux tâches classiques en NLP :
- POS-Tagging : affecte à chaque mot un tag unique qui indique son rôle syntaxique (nom, verbe, adverbe, ..)
- Shallow Parsing : affecte à chaque segment de phrase un tag unique qui indique le rôle de l'élément syntaxique auquel il appartient (groupe nominal, groupe verbal, etc..)
Les fichiers sont ici au format ConLL : un mot par ligne, les phrases sont séparées par un saut de ligne.

Nous allons ici refaire les travaux de l'article NLP almost from scratch, qui consiste à créer un réseau de neurones pour effectuer chaque tâche, puis nous ferons un modèle partagé et enfin un modèle hiearchique.

<img src="MLP.png" style="width:300px;height:450px;">
<caption><center> <u>Figure 1</u>: Modèle simple pour les tâches POS-Tagging et Shallow Parsing</center></caption>

# POS-Tagging
## Import des données

A l'aide des fonctions d'aide, importer les données :

#### Word embedding

In [2]:
import os 
os.chdir(r'C:\Users\Yasmina\Google Drive\Etudes\Paris 1\Master 2\S2\ML avancé\TP\machine-learning-avance\assignment')

In [3]:
wv, word_to_num, num_to_word = pos.load_wv(data+'vocab.txt', data+'wordVectors.txt')
print('wordvector shape is',wv.shape)

wordvector shape is (100003, 50)


On récupère également les tags et on crée les dictionnaires adéquats

#### Création des tags

In [43]:
tagnames = ['ADJ','ADJWH','ADV','ADVWH','CC','CLO',
                'CLR','CLS','CS','DET','DETWH','ET','I','NC',
                'NPP','P','P+D','P+PRO','PONCT','PREF','PRO',
                'PROREL','PROWH','VINF','VPR','VPP','V','VS','VIMP']
num_to_tag = dict(enumerate(tagnames))
tag_to_num = {v:k for k,v in num_to_tag.items()}

In [44]:
print(num_to_tag)
print(tag_to_num)
num_to_tag.items()

{0: 'ADJ', 1: 'ADJWH', 2: 'ADV', 3: 'ADVWH', 4: 'CC', 5: 'CLO', 6: 'CLR', 7: 'CLS', 8: 'CS', 9: 'DET', 10: 'DETWH', 11: 'ET', 12: 'I', 13: 'NC', 14: 'NPP', 15: 'P', 16: 'P+D', 17: 'P+PRO', 18: 'PONCT', 19: 'PREF', 20: 'PRO', 21: 'PROREL', 22: 'PROWH', 23: 'VINF', 24: 'VPR', 25: 'VPP', 26: 'V', 27: 'VS', 28: 'VIMP'}
{'ADJ': 0, 'ADJWH': 1, 'ADV': 2, 'ADVWH': 3, 'CC': 4, 'CLO': 5, 'CLR': 6, 'CLS': 7, 'CS': 8, 'DET': 9, 'DETWH': 10, 'ET': 11, 'I': 12, 'NC': 13, 'NPP': 14, 'P': 15, 'P+D': 16, 'P+PRO': 17, 'PONCT': 18, 'PREF': 19, 'PRO': 20, 'PROREL': 21, 'PROWH': 22, 'VINF': 23, 'VPR': 24, 'VPP': 25, 'V': 26, 'VS': 27, 'VIMP': 28}


dict_items([(0, 'ADJ'), (1, 'ADJWH'), (2, 'ADV'), (3, 'ADVWH'), (4, 'CC'), (5, 'CLO'), (6, 'CLR'), (7, 'CLS'), (8, 'CS'), (9, 'DET'), (10, 'DETWH'), (11, 'ET'), (12, 'I'), (13, 'NC'), (14, 'NPP'), (15, 'P'), (16, 'P+D'), (17, 'P+PRO'), (18, 'PONCT'), (19, 'PREF'), (20, 'PRO'), (21, 'PROREL'), (22, 'PROWH'), (23, 'VINF'), (24, 'VPR'), (25, 'VPP'), (26, 'V'), (27, 'VS'), (28, 'VIMP')])

#### Chargement des données text

Chargement des documents puis création des matrices X_train, y_train, X_test, y_test
le paramètre wsize précise la taille de la fenêtre, choisissez une valeur par défaut parmis (3,5,7)

In [67]:
docs_train = du.load_dataset(data+'train.txt') # liste qui contient les phrases et les tags
X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num, wsize=7) # parcourt la liste et créer les matrices 

In [68]:
docs_train[0]
y_train

array([13,  0, 13, ..., 15, 13, 18])

In [69]:
docs_test = du.load_dataset(data+'test.txt')
X_test, y_test = du.docs_to_windows(
    docs_test, word_to_num, tag_to_num, wsize=7)

In [70]:
y_train = keras.utils.to_categorical(y_train, 29)
y_test = keras.utils.to_categorical(y_test, 29)

In [71]:
y_train
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

On extrait les lignes dans y_test qui contiennent des mots non-présents dans X_train : out of vocabulary.

In [72]:
X_test_oov,Y_test_oov = du.get_oov(X_train,y_train,X_test,y_test)

In [73]:
print("X_train a pour dimension",X_train.shape)
print("X_test a pour dimension",X_test.shape)
print("y_train a pour dimension",y_train.shape)
print("y_test a pour dimension",y_test.shape)

X_train a pour dimension (680238, 7)
X_test a pour dimension (267334, 7)
y_train a pour dimension (680238, 29)
y_test a pour dimension (267334, 29)


In [74]:
print(X_test_oov.shape)
#Y_test_oov.shape

(37073, 7)


### Question :

Expliquer ce que contient X et Y ci-dessus ainsi que leur dimension. (NB: Lorsque l'on définit une fenêtre on est amené à créer un tag pour le début et la fin de phrase afin que les premiers et derniers mots puissent être considérés dans le modèle

### Réponse :

In [75]:
#X représente les tags avec les mots qui correspondent à ces tags, le nombre de lignes correspond au nombre de mots uniques
#dans vocab, le nombre de colonne correspond à la taille de la fenêtre choisie (window size)

#chop your corpus into windows of w words (ex:3 words) where the center word is the one you want to classify
#the other ones are context. Let's call this part of the data X.

#fro each window, get the POS tag for the center word let's call this part of data y

## Création du NN

### Consigne :
Compléter le code ci-dessous pour définir une architecture :
Embed (dim 50) -> Dense -> Dropout -> Predict (Softmax).
N'hésitez pas à consulter l'aide de Keras pour :
    - models.Sequential
    - layers.Embedding
    - layers.Flatten

NB : Prendre garde à bien laisser l'embedding "entrainable" afin que la représentation vectorielle bénéficie aussi de la backprop

In [15]:
model = Sequential()
# Your code Here
e = Embedding(X_train.shape[0], X_train.shape[1], weights=[X_train], input_length=X_train.shape[1], trainable=True)
model.add(e)
#input layer 
model.add(Dense(768, input_shape=(1500,), activation="relu"))
model.add(Dropout(0.2))
# hidden layer
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
# output layer
model.add(Dense(29, activation='softmax'))


In [ ]:
###fenetre = 3

21258/21258 [==============================] - 308s 14ms/step - loss: 1.1530 - accuracy: 0.6505
Epoch 2/3
21258/21258 [==============================] - 306s 14ms/step - loss: 0.8031 - accuracy: 0.7579
Epoch 3/3
21258/21258 [==============================] - 290s 14ms/step - loss: 0.8264 - accuracy: 0.7550

In [15]:
model = Sequential()
# Your code Here
e = Embedding(680238, 3, weights=[X_train], input_length=3, trainable=True)
model.add(e)
#input layer
model.add(Dense(100, input_dim=3, activation="relu"))
model.add(Dense(100, input_dim=3, activation="relu"))
model.add(Dropout(0.25))
# hidden layer
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
# output layer
model.add(Dense(29, activation='softmax'))

In [ ]:
    ### fenetre = 7 avec relu
Epoch 1/3
21258/21258 [==============================] - 597s 28ms/step - loss: 1.1110 - accuracy: 0.6712
Epoch 2/3
21258/21258 [==============================] - 598s 28ms/step - loss: 0.7648 - accuracy: 0.7789
Epoch 3/3
21258/21258 [==============================] - 551s 26ms/step - loss: 0.7961 - accuracy: 0.7733
        
     ### fenetre = 7 avec tanh

In [80]:
model = Sequential()
# Your code Here
e = Embedding(680238, 7, weights=[X_train], input_length=7, trainable=True)
model.add(e)
model.add(Dense(100, input_dim=7, activation="tanh"))
model.add(Dense(100, input_dim=7, activation="tanh"))
model.add(Dropout(0.25))
model.add(Dense(80, activation='tanh'))
model.add(Dense(80, activation='tanh'))
model.add(Dense(80, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

In [ ]:
    ### fenetre = 5 avec tanh
Epoch 1/3
21258/21258 [==============================] - 468s 22ms/step - loss: 0.9490 - accuracy: 0.7221
Epoch 2/3
21258/21258 [==============================] - 431s 20ms/step - loss: 0.6892 - accuracy: 0.7926
Epoch 3/3
21258/21258 [==============================] - 424s 20ms/step - loss: 0.6542 - accuracy: 0.8032

In [58]:

model = Sequential()
# Your code Here
e = Embedding(680238, 5, weights=[X_train], input_length=5, trainable=True)
model.add(e)
model.add(Dense(100, input_dim=5, activation="tanh"))
model.add(Dense(100, input_dim=5, activation="tanh"))
model.add(Dropout(0.25))
model.add(Dense(80, activation='tanh'))
model.add(Dense(80, activation='tanh'))
model.add(Dense(80, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

## Entrainement

### Questions :
- A quoi servent les layers suivants :
    - Flatten()
    - Dropout()
- Combien de paramètres seront appris au cours de l'entrainement? (il existe une commande qui permette de trouver l'architecture du réseau de neurones)

### Réponse

- La layer Flatten() sert à remodèler le tenseur pour avoir la forme qui est égale au nombre d'éléments contenus dans le tenseur sans inclure la dimension du batch

- La layer Dropout() définit de manière aléatoire les unités d'entrée à 0 avec une fréquence de fréquence à chaque étape pendant le temps d'entraînement, ce qui permet d'éviter le surajustement. Les entrées non définies sur 0 sont augmentées de 1 / (taux 1) de sorte que la somme de toutes les entrées reste inchangée.

- le nombre de paramètres appris au cours de l'entraînement est de 3,464,409

In [81]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 7, 7)              4761666   
_________________________________________________________________
dense_90 (Dense)             (None, 7, 100)            800       
_________________________________________________________________
dense_91 (Dense)             (None, 7, 100)            10100     
_________________________________________________________________
dropout_12 (Dropout)         (None, 7, 100)            0         
_________________________________________________________________
dense_92 (Dense)             (None, 7, 80)             8080      
_________________________________________________________________
dense_93 (Dense)             (None, 7, 80)             6480      
_________________________________________________________________
dense_94 (Dense)             (None, 7, 80)            

### Consigne : Compilation
Compléter le code ci-dessous en choisissant l'optimizer RMSprop, et en choissisant la bonne loss (NB on est sur un probléme de classification à 29 modalités). La métrique sera renvoyée par les logs au cours de l'entrainement.

In [82]:
model.compile(loss='categorical_crossentropy',
              optimizer = 'RMSprop',
              metrics=['accuracy'])

### Consigne  : Entrainement
Compléter le code ci-dessous pour entrainer sur le couple X_train, Y_train et en validant sur X_test, Y_test. Vous devriez pouvoir atteindre une accuracy de 93% au bout de 3 itérations. N'oubliez pas que la taille du batch correspond au nombre d'exemples utilisés pour estimer le gradient. Il peut influer sur la convergence.

In [83]:
history = model.fit(X_train,y_train, batch_size=32, epochs=3)

Epoch 1/3
21258/21258 [==============================] - 622s 29ms/step - loss: 0.9157 - accuracy: 0.7312
Epoch 2/3
21258/21258 [==============================] - 605s 28ms/step - loss: 0.6485 - accuracy: 0.8053
Epoch 3/3
21258/21258 [==============================] - 609s 29ms/step - loss: 0.6248 - accuracy: 0.8124


### Question :
- Nous avons décidé de regarder l'accuracy comme métrique. Expliquez en quoi ce choix est discutable.
- On se propose de tester la performance sur des données qui contiennent des mots non-vus lors du train dans la cellule ci-dessous lancer l'évaluation sur X_test_oov,Y_test_oov

- on pourrait regarder le f1 score qui est une moyenne arithmétique de la précision et du recall.

### Réponse :

In [65]:
model.evaluate(X_test_oov,Y_test_oov)

897/897 [==============================] - 3s 3ms/step - loss: 0.6409 - accuracy: 0.8107


[0.6409143209457397, 0.810655415058136]

## Consigne :
- Déterminer quelle est la meilleur taille de fenêtre selon vous (tester les plusieurs hypothèses parmis 3, 5 et 7)
- Reprendre le code pour les différentes hypothèses suivantes avec la taille de fenêtre choisie ci-dessus :
    - 1 embedding random non entrainable
    - 2 embedding pré-entrainer (wv) entrainable
    - 3 embedding pré-entrainer (wv) non entrainable
- Conclure en testant sur X_test_oov et Y_test_oov :
    - Quelle est la méthode que vous choisiriez et pourquoi? 
    - Pouvez-vous expliquer intuitivement les différents résultats ? 

## Réponse

- Après plusieurs tests nous concluons que la meilleure taille de fenêtre est une taille de : 

In [ ]:
# hypothèse 1 : embedding random non entrainable

In [ ]:
# hypothèse 2 : embedding pré-entrainer (wv) entrainable

In [ ]:
# hypothèse 3 : embedding pré-entrainer (wv) non entrainable

Il est possible de sauvegarder le modèle (les poids W) dans un fichier .h5

In [65]:
model.save("my_POStagging_model.h5")

# Shallow Parsing

## Consigne :
- Reprendre la méthodologie ci-dessus et entrainer un modèle de shallow parsing
- Les tags sont ['O','B-NP','I-NP','B-AP','I-AP','B-CONJ',
                'I-CONJ','B-AdP','I-AdP','B-VN','I-VN','B-PP','I-PP','B-UNKNOWN','I-UNKNOWN']
- Les fichiers sont train_chunk.txt et test_chunk.txt
- N'oubliez pas de recréer les tests out-of-vocabulary (X_test_oov et Y_test_oov)
- Conclure sur votre choix du meilleur modèle

#### Word Embedding

In [66]:
wv, word_to_num, num_to_word = pos.load_wv(data+'vocab.txt', data+'wordVectors.txt')
print('wordvector shape is',wv.shape)

wordvector shape is (100003, 50)


#### Création des tags

In [67]:
tagnames =  ['O','B-NP','I-NP','B-AP','I-AP','B-CONJ','I-CONJ',
             'B-AdP','I-AdP','B-VN','I-VN','B-PP','I-PP','B-UNKNOWN','I-UNKNOWN']
num_to_tag = dict(enumerate(tagnames))
tag_to_num = {v:k for k,v in num_to_tag.items()}

#### Chargement des données

In [68]:
docs_train = du.load_dataset(data+'train_chunk.txt') # liste qui contient les phrases et les tags
X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num, wsize=5) # parcourt la liste et créer les matrices 

In [70]:
docs_test = du.load_dataset(data+'test_chunk.txt')
X_test, y_test = du.docs_to_windows(
    docs_test, word_to_num, tag_to_num, wsize=5)

In [71]:
y_train = keras.utils.to_categorical(y_train, 29)
y_test = keras.utils.to_categorical(y_test, 29)

In [72]:
X_test_oov,Y_test_oov = du.get_oov(X_train,y_train,X_test,y_test)

In [73]:
print("X_train a pour dimension",X_train.shape)
print("X_test a pour dimension",X_test.shape)
print("y_train a pour dimension",y_train.shape)
print("y_test a pour dimension",y_test.shape)

X_train a pour dimension (680238, 5)
X_test a pour dimension (267334, 5)
y_train a pour dimension (680238, 29)
y_test a pour dimension (267334, 29)


In [ ]:
print(X_test_oov.shape)
#Y_test_oov.shape

#### Création du réseau de neurones

In [74]:
model = Sequential()
# Your code Here#
e = Embedding(680238, 5, weights=[X_train], input_length=5, trainable=True)
model.add(e)
#model.add(Dense(10, input_dim=5, activation="relu"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

#### Entrainement

In [ ]:
model.summary()

In [75]:
model.compile(loss='categorical_crossentropy',
              optimizer = 'RMSprop',
              metrics=['accuracy'])

In [76]:
history = model.fit(X_train,y_train, batch_size=16, epochs=10)

21258/21258 [==============================] - 443s 21ms/step - loss: 70.5901 - accuracy: 0.2314


#### Conclusion

In [77]:
model.evaluate(X_test_oov,Y_test_oov, batch_size=16, epochs=10)

897/897 [==============================] - 1s 1ms/step - loss: 13.0831 - accuracy: 0.2510


[13.083104133605957, 0.2510192096233368]

# Multi-task learning

## Full multi tagged (POS tagging + Shallow parsing)

En deep learning, il est possible et parfois même recommandé d'entrainer un réseau de neurones sur plusieurs tâches en même temps. Intuitivement on se dit que l'apprentissage de représentation sur une tâche devrait pouvoir aider sur une autre tâche. Cela permet en outre de disposer de plus de données par exemple et/ou d'avoir un modèle plus robuste, plus précis.

Dans cette partie, on va s'appuyer sur la classe Model de Keras https://keras.io/getting-started/functional-api-guide/. Plutôt que d'ajouter les layers, il s'agit plutôt de voire chaque layer comme une fonction. Il s'agit alors d'enchainer les fonctions.

<img src="mtl_images.png" style="width:300px;height:450px;">
<caption><center> <u>Figure 1</u>: Un exemple d'architecture pour apprendre 3 tâches </center></caption>

#### Embedding

In [ ]:
wv, word_to_num, num_to_word = pos.load_wv(
      data+'vocab.txt', data+'wordVectors.txt')
print('wordvector shape is',wv.shape)

#### Création des tags

In [ ]:
postagnames = ['ADJ','ADJWH','ADV','ADVWH','CC','CLO',
                'CLR','CLS','CS','DET','DETWH','ET','I','NC',
                'NPP','P','P+D','P+PRO','PONCT','PREF','PRO',
                'PROREL','PROWH','VINF','VPR','VPP','V','VS','VIMP']
num_to_postag = dict(enumerate(postagnames))
postag_to_num = {v:k for k,v in num_to_postag.items()}

In [ ]:
chunktagnames = ['O','B-NP','I-NP','B-AP','I-AP','B-CONJ',
                'I-CONJ','B-AdP','I-AdP','B-VN','I-VN','B-PP','I-PP','B-UNKNOWN','I-UNKNOWN']
num_to_chunktag = dict(enumerate(chunktagnames))
chunktag_to_num = {v:k for k,v in num_to_chunktag.items()}

#### Import des données

In [ ]:
# Load the training set
docs = du.load_dataset(data+'train.txt')
X_train, pos_train = du.docs_to_windows(
    docs, word_to_num, postag_to_num, wsize=5)
docs = du.load_dataset(data+'train_chunk')
X_train, chunk_train = du.docs_to_windows(
    docs, word_to_num, chunktag_to_num, wsize=5)

In [ ]:
docs = du.load_dataset(data+'test.txt')
X_test, pos_test = du.docs_to_windows(
    docs, word_to_num, postag_to_num, wsize=5)
docs = du.load_dataset(data+'test_chunk')
X_test, chunk_test = du.docs_to_windows(
    docs, word_to_num, chunktag_to_num, wsize=5)

#### Mise en forme pour l'apprentissage

In [ ]:
X_pos_train=X_train[:400000,:]

X_chunk_train=X_train[-400000:,]

In [ ]:
Y_pos_train = keras.utils.to_categorical(pos_train[0:400000,], 29)
Y_pos_test = keras.utils.to_categorical(pos_test, 29)
Y_chunk_train = keras.utils.to_categorical(chunk_train, 15)
Y_chunk_test = keras.utils.to_categorical(chunk_test[-400000:,], 15)

#### Résumé
Jusqu'à présent on dispose de :
- wv qui contient un embedding
- X_pos_ et Y_pos_ qui contiennent respectivement les mots et le tag sur train et test pour le pos tagging
- X_chunk_ et Y_chunk_ qui contiennent respectivement les mots et le tag sur train et test pour le shallow parsing
- X_pos et X_chunk n'ont aucune fenêtre en commun

### Création du réseaux de neurones commun aux deux tâches

#### Consigne
Utiliser la même architecture que précedemment pour construire le réseau de neurones communs : shared_nn

In [ ]:
shared_nn=Sequential()

#### Consigne
Définir les inputs du réseaux de neurones communs à l'aide de la fonction Input. Ce sont les input layer de notre architecture, chacun correspond à une tâche.

In [ ]:
pos_input = keras.Input(shape=(784,))
chunk_input = keras.Input(shape=(784,))

#### Consigne
- Définir pos_representation et chunk_representation comme les images respectibes de chaque input par shared_nn
- Définir pos_target et chunk_target comme l'image des représentations ci-dessus par des layers dense avec activation softmax (cf  la fonction Dense)

Rappel : la fonction softmax va calculer $n$ softmax, où $n$ est le nombre de classe en suivant la formule :
$$\sigma(z)_j = \frac{\exp(z_j)}{\sum_{i}\exp(z_i)},$$
avec $z$ le vecteur en sortie du réseau de neurones et $z_i$ sa $i$-ième composante

In [ ]:
pos_representation =
chunk_representation =

pos_target = 
chunk_target = 

### Entrainement et conclusion

#### Consigne 
Jusqu'ici on a juste défini la succession d'opération, il s'agit maintenant de créer le modèle en utilisant la classe modèle de Keras

In [ ]:
model= Model(inputs=,outputs=)

#### Consigne
- compiler le modèle (NB : il y a une loss pour chaque tâche)
- afficher l'architecture
- entrainer le modèle pour le nombre d'époque suffisant en validant sur X_test et Y_test et une taille de batch de 128
- Conclure en comparant avec les performances précédentes

In [ ]:
model.compile()

In [ ]:
model.fit()

## Hierarchical learning
Une autre façon de faire du multi-task consiste à construire une architecture en cascade où les tâches n'interviennent pas à la même profondeur du réseau de neurones.

### Consigne :
- En vous inspirant de la partie précedente, entrainer un modèle en cascade de type : 
                              POS
                            /
EMBEDDING - DENSE - DROPOUT 
                            \
                              DENSE - DROPOUT - CHUNK
                              
Autrement dit, dans cette approche, on pré-suppose que  
- EMBEDDING - DENSE - DROPOUT apprend une représentation suffisante pour prédire le POS-TAG
- plus de layer sont nécessaires pour le Shallow parsing
- la représentation intermédiaire EMBEDDING - DENSE - DROPOUT est un bon point de départ pour le shallow parsing

# Question optionnelle 
- Comment feriez vous, en utilisant la classe Model, pour pouvoir extraire l'embedding tuné sur le modèle pour un mot donné? Tenter de l'implémenter.